## Normalfördelning
### Kurvan är lägre och bredare vid större spridning (standard deviation)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# En normalfördelning med 100,000 värden med medelvärde=0 och spridning=1.0
distr1 = np.random.normal(loc=0.0, scale=1.0,size=100000)
# En normalfördelning med 100,000 värden med medelvärde=0 och spridning=1.5
distr2 = np.random.normal(loc=0.0, scale=1.5, size=100000)
data = [distr1, distr2]
# Vi plottar normalfördelningarna som histogram med 100 intervall ("hinkar")
# Ger 100 staplar där höjden av varje stapel är ett mått på hur många värden
# av fördelningen som finns i det intervallet.
_,_,_ = plt.hist(data, 100, density=True, histtype='stepfilled', alpha=0.6)
plt.legend(["dev=1.5", "dev=1.0"], loc ="lower right")
plt.show()

# Definition av VAE autoencoder

### Modell för VAE encoder

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

latent_dim = 2

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var], name="encoder")

In [ ]:
encoder.summary()

### Lager för sampling (Sampler)

In [ ]:
import tensorflow as tf

class Sampler(layers.Layer):
    def call(self, z_mean, z_log_var):
        batch_size = tf.shape(z_mean)[0]
        z_size = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch_size, z_size))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

### Modell för VAE decoder

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2D(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

In [ ]:
decoder.summary()

### VAE modell för träning med encoder, sampler och decoder

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.sampler = Sampler()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var = self.encoder(data)
            z = self.sampler(z_mean, z_log_var)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            total_loss = reconstruction_loss/30. + tf.reduce_mean(kl_loss)
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "total_loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

### Om du inte vill träna VAE (tar en timme) ladda från disk

In [ ]:
from keras.models import load_model
from tensorflow import keras
import numpy as np
decoder = load_model("mnist_decoder_60ep")
encoder = load_model("mnist_encoder_60ep")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
mnist_labels = np.concatenate([y_train, y_test], axis=0)

In [ ]:
mnist_labels.shape

### Träna VAE - hoppa över om du laddade från disk

In [ ]:
import numpy as np

# antal epoker
neps=30

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
mnist_labels = np.concatenate([y_train, y_test], axis=0)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam(), run_eagerly=True)
vae.fit(mnist_digits, epochs=neps, batch_size=128)
dir = "mnist_encoder_" + str(neps) + "ep"
#encoder.save(dir)
dir = "mnist_decoder_" + str(neps) + "ep"
#decoder.save(dir)

### Vilken noggrannhet får vi?

In [ ]:
# 30 ep -> 66%
# 60 ep -> 77% BÄST! Används i fortsättningen...
# 90 ep -> 75%
num = 1000
err = 0
for ix in range(num):
    enc = encoder.predict(mnist_digits[ix][np.newaxis])
    res = decoder.predict(enc[0])
    vae= (clmodel.predict(res[0][np.newaxis]).argmax())
    orig = mnist_labels[ix]
    if vae!= orig: err = err + 1
    plt.show()
print("Total:", num, " errors", err, " accuracy:", 100*(num-err)/num, "%")

#### Några arrayer och tensorer

In [ ]:
print(type(mnist_digits))
print(mnist_digits.shape)
print(mnist_digits[0].shape)
print(mnist_digits[0][np.newaxis].shape)
# z_mean + z_log_var. 2 värden var.
enc = encoder(mnist_digits[0][np.newaxis])
print(type(enc), enc)
print(enc[0], enc[1])

### Hur ser siffrorna ut före och efter VAE?

In [ ]:
from keras.models import load_model
clmodel = load_model("mnist_classify")

In [ ]:
import matplotlib.pyplot as plt
for ix in range(20):
    enc = encoder(mnist_digits[ix][np.newaxis])
    print("---------------------------------------------")
    print(enc[0])
    print(enc[1])
    res = decoder(enc[0])
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(mnist_digits[ix], cmap='gray')
    orig = mnist_labels[ix]
    plt.subplot(1, 2, 2)
    plt.imshow(res[0], cmap='gray')
    vae =(clmodel.predict(res[0][np.newaxis]).argmax())
    print(orig, "after vae:", vae)
    plt.show()

#### Hur kodas siffrorna i planet? Vi kan få en uppfattning genom att ta z från bara z_mean och strunta i tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
import matplotlib.pyplot as plt
encoder = load_model("mnist_encoder_60ep")
num = 10000
arr = np.zeros((num, 3))
for ix in range(num):
    enc = encoder(mnist_digits[ix][np.newaxis])
    arr[ix, 0] = mnist_labels[ix]
    arr[ix, 1] = (enc[0].numpy())[0][0]
    arr[ix, 2] = (enc[0].numpy())[0][1]

_, ax = plt.subplots(figsize=(10, 7))
for i in range(10):
    x = np.zeros((num, 3)) 
    y = np.zeros((num, 3))
    nd = 0
    mean_x = 0.
    mean_y = 0.
    for ix in range(num):
        if i == arr[ix,0]:
            x[nd] = arr[ix, 1]
            y[nd] = arr[ix, 2]
            mean_x += arr[ix, 1]
            mean_y += arr[ix, 2]
            nd = nd + 1

    mean_x = mean_x / (nd - 1)
    mean_y = mean_y / (nd - 1)  
    plt.scatter(x[:nd], y[:nd])
    ax.text(mean_x, mean_y, str(i), fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
plt.grid(visible=True)
plt.legend(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], loc ="lower right")

#### Högre siffror skymmer lägre - här en och en

In [ ]:
#encoder = load_model("mnist_encoder_90ep")
num = 10000
arr = np.zeros((num, 3))
for ix in range(num):
    enc = encoder(mnist_digits[ix][np.newaxis])
    arr[ix, 0] = mnist_labels[ix]
    arr[ix, 1] = (enc[0].numpy())[0][0]
    arr[ix, 2] = (enc[0].numpy())[0][1]

#_, ax = plt.subplots(figsize=(10, 7))
for i in range(10):
    _, ax = plt.subplots(figsize=(10, 7))
    x = np.zeros((num, 3)) 
    y = np.zeros((num, 3))
    nd = 0
    mean_x = 0.
    mean_y = 0.
    for ix in range(num):
        if i == arr[ix,0]:
            x[nd] = arr[ix, 1]
            y[nd] = arr[ix, 2]
            mean_x += arr[ix, 1]
            mean_y += arr[ix, 2]
            nd = nd + 1

    mean_x = mean_x / (nd - 1)
    mean_y = mean_y / (nd - 1)  
    plt.scatter(x[:nd], y[:nd], alpha=0.5)
    ax.text(mean_x, mean_y, str(i), fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

**Sampling a grid of images from the 2D latent space**

In [ ]:
def decode(x, y):
    res = decoder(np.array([[x, y]]))
    plt.imshow(res[0], cmap='gray')

In [ ]:
decode(0, 0)

In [ ]:
decode(0, -1)

### image_grid

In [ ]:
def image_grid(x_min, x_max, y_min, y_max):
    import matplotlib.pyplot as plt
    import numpy as np

    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))

    grid_x = np.linspace(x_min, x_max, n)
    grid_y = np.linspace(y_min, y_max, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[
                i * digit_size : (i + 1) * digit_size,
                j * digit_size : (j + 1) * digit_size,
            ] = digit

    plt.figure(figsize=(15, 15))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.axis("on")
    plt.imshow(figure, cmap="Greys_r")

In [ ]:
image_grid(-1, 1, -1, 1)

### digit_grid

In [ ]:
def digit_grid(x_min, x_max, y_min, y_max):
    import numpy as np

    n = 30
    row = np.zeros(n, dtype=np.int8)

    grid_x = np.linspace(x_min, x_max, n)
    grid_y = np.linspace(y_min, y_max, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(28, 28)
            digit = (clmodel.predict(digit[np.newaxis]).argmax())
            row[j] = digit
        print(row)


In [ ]:
digit_grid(-1, 1, -1, 1)

### Animering

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# backend för att visa animering
matplotlib.use( 'tkagg' )

x_min = -1.
x_max = 1.
x_step = 0.07
y_min = -1.
y_max = 1.
y_step = 0.07

# börja i punketn (x_min, y_max)
y = y_max
fig, ax = plt.subplots()
i = 0
while y > y_min:
    x = x_min
    while x < x_max:
        ax.cla()
        res = decoder(np.array([[x, y]]))
        ax.imshow(res[0], cmap='gray')
        ax.set_title("({:.2f}, {:.2f})".format(x,y))
        plt.pause(0.1)
        # gå framåt i raden
        x = x + x_step
        i = i + 1
    # gå ner en rad
    y = y - y_step
    x = x_max
    while x > x_min:
        ax.cla()
        res = decoder(np.array([[x, y]]))
        ax.imshow(res[0], cmap='gray')
        ax.set_title("({:.2f}, {:.2f})".format(x,y))
        plt.pause(0.1)
        # gå bakåt i raden
        x = x - x_step
        i = i + 1
    y = y - y_step
plt.close()
# återställ till vanlig backend för matplotlib
matplotlib.use('module://ipykernel.pylab.backend_inline')